In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136032
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-136032


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "computecluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

ps = RandomParameterSampling( {
        "--C" : choice(1,2,3,4),
        "--max_iter" : choice(10,50,100,200)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")  
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', 
              entry_script = 'train.py', 
              compute_target = cpu_cluster)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = ps, 
                                     policy = policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_concurrent_runs = 5)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_96a5d851-48ec-44b3-86c2-526ba9386d97
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_96a5d851-48ec-44b3-86c2-526ba9386d97?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-136032/workspaces/quick-starts-ws-136032

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-25T21:49:12.231535][API][INFO]Experiment created<END>\n""<START>[2021-01-25T21:49:12.990482][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n"<START>[2021-01-25T21:49:13.2938829Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-25T21:49:13.325166][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"


In [33]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy: ', best_run_metrics['Accuracy'])
print('Regularization Rate: ',parameter_values[1])
print('Number of iterations: ',parameter_values[3])

best_run.download_file("outputs/model.joblib", "model.joblib")

Best Run Id:  HD_96a5d851-48ec-44b3-86c2-526ba9386d97_2
Accuracy:  0.9118361153262519
Regularization Rate:  4
Number of iterations:  100


In [21]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_96a5d851-48ec-44b3-86c2-526ba9386d97_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [34]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds_new = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [36]:
from train import clean_data
import pandas as pd 

# Use the clean_data function to clean your data.
x, y = clean_data(ds_new)
df_combine = pd.concat([x, y], axis=1)

In [37]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=df_combine,
    label_column_name="y",
    n_cross_validations=5)

In [38]:
# Submit your automl run

### YOUR CODE HERE ###
automl_exp = Experiment(ws, 'automl')
automl_run = automl_exp.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |1                                |32950                                 |
+---------------------------------+----------

{'runId': 'AutoML_ff2aa364-53fd-48a0-b49c-097b0ad5c121',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-25T22:32:23.672906Z',
 'endTimeUtc': '2021-01-25T23:05:48.355007Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-136032","workspace_name":"quick-starts-ws-136032","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":2,"featurizat

In [39]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

best_run.download_file('outputs/model.pkl', 'automl_best_model.pkl')

Run(Experiment: automl,
Id: AutoML_ff2aa364-53fd-48a0-b49c-097b0ad5c121_32,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.2442105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                 

In [40]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_ff2aa364-53fd-48a0-b49c-097b0ad5c121_32,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
cpu_cluster.delete()